In [7]:
%load_ext jupyter_black
%matplotlib inline

backend = "torch"

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


A brief tutorial for how to use agent.Discrete class from pybefit to define a computational model of behaviour

In [21]:
import numpy as np
import torch
import jax.numpy as jnp
from jax import nn, random

if backend == "torch":
    tensor = torch
    one_hot = torch.nn.functional.one_hot
    sigmoid = torch.nn.functional.sigmoid

    class Cat(object):
        def __init__(self, seed=0):
            torch.manual_seed(seed)

        def __call__(self, logits):
            dist = torch.distributions.Categorical(logits=logits)
            return dist.sample()

else:
    tensor = jnp
    one_hot = nn.one_hot
    sigmoid = nn.sigmoid

    class Cat(object):
        def __init__(self, seed=0):
            self.key = random.PRNGKey(seed)

        def __call__(self, logits):
            self.key, key = random.split(self.key)
            return random.categorical(key, logits)


categorical = Cat(seed=0)

In [2]:
# One would access this class using 'from pybefit.agents import Discrete'.
# Note that each of the methods has to be implemented for simulations and inference to be possible with using pybefit classes

class Discrete(object):
    """Agent with discrete and finite number of actions.
    """
    def __init__(self, runs, blocks, trials, na, ns, no):
        
        self.runs = runs  # number of independent runs of the experiment or agents/subjects
        self.nb = blocks  # number of experimental blocks
        self.nt = trials  # number of trials in each block
        
        self.na = na  # number of actions
        self.ns = ns  # number of states
        self.no = no  # number of outcomes

    @property
    def num_params(self):
        """Return the number of model parameters
        """
        raise NotImplementedError
        
    def set_parameters(self, *args, **kwargs):
        """Set free model parameters.
        """
        raise NotImplementedError
        
    def update_beliefs(self, block, trial, *args, **kwargs):
        """Update beliefs about hidden states given some sensory stimuli and action outcomes.
        """        
        raise NotImplementedError

    def planning(self, block, trial, *args, **kwargs):
        """Compute choice probabilities in current block and trial.
        """
        raise NotImplementedError
        
    def sample_responses(self, block, trial, *args, **kwargs):
        """Generate responses given response probability.
        """
        raise NotImplementedError

We will implement a simple agent solving a static multi armed bandit task based on the UCB model

In [3]:
class MABAgent(Discrete):

    def __init__(self, runs=1, blocks=1, trials=1, num_arms=2):
        # define bernoulli bandit with two outcomes (0, 1) for each arm
        super().__init__(runs, blocks, trials, num_arms, num_arms, 2)

    @property
    def num_params(self):
        return 2
    
    def set_parameters(self, z):
        self.lr = sigmoid( z[..., -1] )  # learning rate
        self.beta = tensor.exp( z[..., -1] )  # exploration strength

        self.q = tensor.zeros()  # q values
        self.count = tensor.zeros()  # response count

    def update_beliefs(self, block, trial, response_outcome):
        
        # encode reponses as zero/one array where one is assigned to the chosen arm and zero to all other arms
        response = one_hot(response_outcome[0], self.na)

        # add one dimension to the right to outcomes to match dimensionality of responses
        outcome = response_outcome[1][..., None] 


        alpha = self.lr / (self.count + 1)

        # implements self.q[..., response] += alpha * (outcome - self.q[..., response])
        self.q += alpha * response * (outcome[..., None] - self.q)
        self.count += response

    def planning(self, block, trial, *args, **kwargs):
        
        self.logits = self.q + self.beta * tensor.sqrt( tensor.log(trial + 1)/(self.count + 1e-6) )
        return self.logits 
        
    def sample_responses(self, block, trial, *args, **kwargs):

        return categorical(self.logits)


Next we will define a task environmnet for the multi armed bandit

In [27]:
# As with the agent implementation we start with a base Task class which is imported as "from pybefit.task import Task"

class Task(object):
    def __init__(self, nsub, blocks, trials):
        self.blocks = blocks  # number of experimental blocks
        self.trials = trials  # number of trials
        self.nsub = nsub  # number of subjects

    def get_offer(self, block, trial, *args, **kwargs):
        """Define an offer for a current block, trial pair that defines a unique stimuli"""
        
        return None

    def update_environment(self, block, trial, *args, **kwargs):
        """Generate stimuli for task's current block and trial"""

        raise NotImplementedError


class MABTask(Task):
    def __init__(self, outcomes):
        blocks, trials, nsub, _ = outcomes.shape
        super().__init__(nsub, blocks, trials)
        self.outcomes = outcomes

    def update_environment(self, block, trial, responses):
        return self.outcomes[block, trial, range(self.nsub), responses]


blocks = 10
trials = 20
num_arms = 3
num_subjects = 100

probs = np.random.dirichlet(np.ones(num_arms), size=(blocks, trials, num_subjects))

outcomes = tensor.ones(1) * np.random.binomial(1, probs)
print(outcomes.dtype)

mab_task = MABTask(outcomes)
mab_task.update_environment(0, 0, tensor.zeros(num_subjects, dtype=tensor.long))

torch.float64


tensor([0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
        1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)